In [28]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

import matplotlib.pyplot as plt
import pickle

In [29]:
def train_model(train_df, train_target):
    model = RandomForestClassifier( n_estimators=100, n_jobs=-1, random_state=42 ).fit(train_df, train_target)
    print("training is complete")
    return model

In [30]:
def save_model(model, dv):
    with open('model.pkl', 'wb') as f_out:
        pickle.dump((dv,model), f_out)

In [31]:
import mlflow


mlflow.set_tracking_uri("sqlite:///final_project.db")
mlflow.set_experiment("car-prediction-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='car-prediction-experiment', tags={}>

In [32]:
from prediction_service.predict import make_encoding
df = pd.read_csv(r"../data/car_data.csv")
df = make_encoding(df)
X = df[["Gender", "Age", "AnnualSalary"]]
y = df["Purchased"]

FileNotFoundError: [Errno 2] No such file or directory: 'rf.pkl'

In [64]:
train_df, test_df, train_target, test_target = train_test_split(X, y, test_size = 0.33)

In [66]:
dv = DictVectorizer()

In [67]:
train_dicts = train_df.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = test_df.to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [68]:
model = train_model(X_train, train_target)

training is complete


In [27]:
with open('prediction_service/rf.pkl', 'wb') as f_out:
    pickle.dump((dv, model), f_out)

NameError: name 'dv' is not defined

In [53]:
dv.get_feature_names_out()

array(['Age', 'AnnualSalary', 'Gender'], dtype=object)

In [54]:
train_df

,Gender,Age,AnnualSalary
453,0,47,50000
191,1,50,66500
14,0,42,108000
239,0,27,31000
685,1,36,40500
...,...,...,...
382,1,19,70000
407,1,20,71500
679,1,35,55000
46,0,49,76500


In [14]:
df[df["User ID"] == 191]

,User ID,Gender,Age,AnnualSalary,Purchased
98,191,Male,30,87000,0


In [22]:
import pyarrow as pa
import uuid
import pandas as pd
table = pd.read_csv("../data/car_data.csv")
table = pa.Table.from_pandas(table, preserve_index=True)
data  = table.to_pylist()


In [27]:
duration = int(1)
with open("target.csv", 'w') as f_target:
    for row in data:
        f_target.write(f"{duration}\n")

In [46]:
with open("target.csv", 'w') as f_target:
    for row in data:
        print(row)

[
{
"
U
s
e
r
 
I
D
"
:
3
8
5
,
"
G
e
n
d
e
r
"
:
"
M
a
l
e
"
,
"
A
g
e
"
:
3
5
,
"
A
n
n
u
a
l
S
a
l
a
r
y
"
:
2
0
0
0
0
,
"
P
u
r
c
h
a
s
e
d
"
:
0
}
,
{
"
U
s
e
r
 
I
D
"
:
6
8
1
,
"
G
e
n
d
e
r
"
:
"
M
a
l
e
"
,
"
A
g
e
"
:
4
0
,
"
A
n
n
u
a
l
S
a
l
a
r
y
"
:
4
3
5
0
0
,
"
P
u
r
c
h
a
s
e
d
"
:
0
}
,
{
"
U
s
e
r
 
I
D
"
:
3
5
3
,
"
G
e
n
d
e
r
"
:
"
M
a
l
e
"
,
"
A
g
e
"
:
4
9
,
"
A
n
n
u
a
l
S
a
l
a
r
y
"
:
7
4
0
0
0
,
"
P
u
r
c
h
a
s
e
d
"
:
0
}
,
{
"
U
s
e
r
 
I
D
"
:
8
9
5
,
"
G
e
n
d
e
r
"
:
"
M
a
l
e
"
,
"
A
g
e
"
:
4
0
,
"
A
n
n
u
a
l
S
a
l
a
r
y
"
:
1
0
7
5
0
0
,
"
P
u
r
c
h
a
s
e
d
"
:
1
}
,
{
"
U
s
e
r
 
I
D
"
:
6
6
1
,
"
G
e
n
d
e
r
"
:
"
M
a
l
e
"
,
"
A
g
e
"
:
2
5
,
"
A
n
n
u
a
l
S
a
l
a
r
y
"
:
7
9
0
0
0
,
"
P
u
r
c
h
a
s
e
d
"
:
0
}
,
{
"
U
s
e
r
 
I
D
"
:
8
4
6
,
"
G
e
n
d
e
r
"
:
"
F
e
m
a
l
e
"
,
"
A
g
e
"
:
4
7
,
"
A
n
n
u
a
l
S
a
l
a
r
y
"
:
3
3
5
0
0
,
"
P
u
r
c
h
a
s
e
d
"
:
1
}
,
{
"
U
s
e
r
 
I
D
"
:
2
1
9
,
"
G
e
n
d
e
r
"
:
"
F
e
m
a
l
e
"
,
"
A
g
e
"
:
4
